---
> `**PLEASE NOTE**: The project undertaken below is not for commercial use, as there are some assumptions that have been made which are flawed and not fit for business purposes. If there is interest to do an indepth study on this subject please contact me via linkedin (https://www.linkedin.com/in/johan-strydom-04331830/).`
---
# Final Project | Commute or Relocate... that is the question
## Project description
### Problem definition

Johannesburg and Pretoria are situated in the heart of Gauteng, a province in South Africa. These two cities combined have the highest populations and number of job opportunities in South Africa. They are also relatively close to one another (+-50km apart centre to centre).

With the cities being spread over a large geographical area, many people tend to only operate in their familiar region of their own city that they know and trust. There are a few of misconceptions between these two cities caused by this reason and historic prejudices. The effect is that when a job opportunity comes along and is in a different area, or city, you do not relocate, but rather extend your daily commute.

Recently, there has been a warning from the government that this commute could increase considerably in the next few years, due to infrastructure development not being supported by the residents. (Please see: https://www.timeslive.co.za/news/south-africa/2018-11-12-get-ready-for-a-six-hour-commute-from-joburg-to-pretoria/)

### Intended audience
The main idea with this project is to analyse the suburbs of Johannesburg and Pretoria, using their municipal regions, house sales information as well as FourSquare venue data to cluster similar neighbourhoods. With this information it would be possible for people moving between the cities, to start considering relocation to similar neighbourhoods, as an alternative to long daily commutes.

### Scope
A problem with using clustering is that it is not clear why specific neighbourhoods would be similar, but with manual analysis it can be determined.

For the purpose of this analysis I will simply cluster the different neighbourhoods and display them on a physical map.

I will then conclude with some findings from the analysis.

## Data aquisition
### Johannesburg and Pretoria municipal regions
There was a census done in the year 2011, where the findings were quite well documented on the below link:
> https://census2011.adrianfrith.com/place/7

From this link I will only extract the official region detail and suburb information. This information I will then be used to get the specific latitude and longitudes from the HERE geocoder API.

### House sales information
There are a few good online websites where houses are listed for rent or for sale. I have identified Private Property (see link below) as the website to be used for my data gathering as it has a clean design in the website layout:
> https://www.privateproperty.co.za/

There is an abundance of data on this site and houses that are uploaded most recently are presented first. In order to level the playing fields I will sort the properties from cheapest to most expensive. I will only include the first 10 or less houses to be presented on the first page after being sorted (i.e. only consider the cheapest properties for each suburb). The following metrics will be extracted:
1. Property for sale prices
2. Number of bed rooms on the property

These two attributes are chosen to ensure the following:
1. The property being considered is for residential use (i.e. a house, townhouse, flat, etc.)
2. This creates the lowest barrier to entry for a relocating employee
3. Gives an indication of the size of the cheapest residential property, and by extension if the area is suited for small or large families or for single employees

### FourSquare data
The venue data for the surrounds will be retrieved from the FourSquare API. This data will give good insight into the facilities and businesses that are available in the nearby area for each location. I will be using the venue_category from FourSquare to get an indication of the different venues surrounding the suburb.

# Data acquisition
## Creating the base
As stated in the previous sections, I need the following data:
1. Municipal regions and suburbs
2. House sales data
3. Latlongs for the areas
4. Venue data ontop of these

The first step in this process is to collect the base: list of all the region and their constituting suburbs.

Below is the collection of packages I used for the project.

In [1]:
## Web scraping
# Navigating
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
# HTML parsing
from bs4 import BeautifulSoup
# RESTful API requests
import requests

## Data processing
import pandas as pd
import numpy as np
import collections
import math
import json  #data persistance
from sklearn.cluster import KMeans #Clustering

## Python Pattern Matching
# import pmatch
# import ast

## Progress bar creation
import operator
from IPython.display import clear_output

#Data Visualization
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

### 1. Municipal region information
The following is the first of two parts where I used the Selenium package to gather data for this project.
Th site used was:
> https://census2011.adrianfrith.com/place/7

The below functions scrapes the data in a recursive function. There are some fundamental assumptions that have been made in using this data:
1. _The area names are spelled correctly_ -- when using the results in the house sales site I found that there are discrepancies in spelling some names between the sites (e.g. Census: Theresapark vs PrivateProperty: Theresa Park). I have assumed that the Census data has the correct spelling.
2. _All suburbs included under correct regions_ -- when inspecting results it was found that some suburbs had the same name between the two cities (e.g. Roodepoort). This is addressed when finalising the census data.

The final output is a dictionary containing lists:
1. __Keys__  -- City of Tshwane (a.k.a. Pretoria), City of Johannesburg and their immediate constituents (119 items)
2. __Values__ -- the key's immediate constituents. I.e. each element of the lists of City of Tshwane and Johannesburg became a key with their immediate constituent suburbs

In [2]:
def driverLevel(test_str):
    """
    DOCSTRING: This simply checks whether to break or proceed with the current  area flow
    """
    title = test_str.lower()
    if ('province' in title):
        return True
    elif ('metropolitan' in title):
        return True
    elif ('main place' in title):
        return False
    else: False
        
def dictionaryCreator(driver,search_list):
    """
    DOCSTRING: This function generates a dictionary with the list item as key
    and a list of its constituent lower level areas
    INPUTS: receives the current browser and a list to run through
    OUTPUTS: two dictionary items per list item;
        1) {Item : list[item_sub_areas]}
            (NOTE: first item in the list is the title of the page AFTER the link is clicked.
            This will be used when constructing the final DF, as well as
            control flow for the consolidated region populator)
        2) {Item+'_url' : 'url_after_item_click'}
    """
    init_dict = {}
    for it in search_list:
        try:
            elem = driver.find_element_by_link_text(it)
        except:
            driver.back()
            elem = driver.find_element_by_link_text(it)
            
        dict_key = elem.text

        url_key = it + '_url'
        elem.click()
        curr_url = driver.current_url
        url_dict = {url_key: curr_url}

        init_dict.update(url_dict)

        try:
            region_table = driver.find_element_by_xpath('/html/body/div/div[2]/div[2]/table/tbody')
            links = region_table.find_elements_by_tag_name('a')
            links_list = [el.text for el in links]
        except:
            links_list = [it]
            
        links_list.insert(0,driver.title) # this is where the title is added
        dict_key = it
        it_dict = {dict_key: links_list}

        init_dict.update(it_dict)
            
    return init_dict
        
def regionPopulator(driver,search_list):
    """
    DOCSTRING: This function does the control flow for this site. It completes the process recursively.
    Therefore the 'break' in the else statement is crucial.
    """
    area_dict = dictionaryCreator(search_list = search_list,driver = driver)
    check_dict = {se:[area_dict[se][0],area_dict[se+'_url']] for se in search_list}
    for i in check_dict.keys():
        if driverLevel(check_dict[i][0]):
            driver.get(check_dict[i][1])
            new_s_list = area_dict[i][1:]
            new_area_dict = regionPopulator(driver = driver,search_list = new_s_list)
            area_dict.update(new_area_dict)
        else:
            break    
    return area_dict
    
## Main code
# city_list = ['City of Tshwane','City of Johannesburg']
# home_page = "https://census2011.adrianfrith.com/place/7"
# browser = webdriver.Chrome()
# browser.get(home_page)
# region_d = regionPopulator(search_list = city_list,driver = browser)
# region_dict = {se:region_d[se] for se in filter(lambda x: '_url' not in x,region_d.keys())}
# browser.close()

### 2. House sales information
After I had scraped the different regions for Johannesburg (a.k.a. City of Johannesburg) and Pretoria (a.k.a. City of Tshwane) I used these regions to search on an open property sales site to get the house sales data I required:
> https://www.privateproperty.co.za/

With a poor internet connection the below query ran for 2hours to complete. I therefore wrote the below output to two JSON files:
1. region_dict {area_key:[list constituent areas]} -> municipalRegions.json
2. region_dict_df {area_key: sales info DataFrame} -> propertyDataFrames.json

In [3]:
def enterLocation(driver,area_name):
    try:
        elem = driver.find_element_by_name("locationPhrase")
        elem.clear()
    except:
        elem = driver.find_element_by_xpath('//*[@id="siteSearchForm"]/div/div/span/select')
        actions = ActionChains(driver)
        actions.send_keys(Keys.TAB)
        elem = driver.switch_to.active_element
        elem.clear()
    finally:
        elem.send_keys(area_name)
        elem.send_keys(Keys.RETURN)
    return driver

def locationDirector(driver,area_name):
    start_url = driver.current_url
    try:
        sort_elem = driver.find_element_by_link_text("Lowest Price")
        sort_elem.click()
    except:
        elem_list = driver.find_elements_by_partial_link_text(area_name)
        new_list = list(filter(lambda x: start_url not in x.get_attribute('href'),[i for i in elem_list]))
        multi_elem = new_list[0]
        multi_elem.click()
        locationDirector(driver = driver,area_name = area_name)
    finally:
        return driver

def propertyDataPopulator(area_names):
    init_list = []

    browser = webdriver.Chrome()
    browser.get("https://www.privateproperty.co.za/")
    assert "Property" in browser.title
    for area in area_names:
        print('Now @ [{}]'.format(area))
        
        browser = enterLocation(driver = browser,area_name = area)
                
        if ("no result" in browser.title.lower()):
            pass
        else:
        
            browser = locationDirector(driver = browser, area_name = area)
            
            soup = BeautifulSoup(browser.page_source,'lxml')
            srch_res = soup.find_all(class_='infoHolder')

            # initialising the item counter, only looking at first 10 (or less) items per area
            counter = 0
            for i,elem in enumerate(srch_res):
                # this brings back the first nested div element in elem
                # this I use to check if the listing is relevant to the area
                title_tmp = elem.div.text

                # I check if there is a number of beds in the description
                # if so I assign the number to beds_tmp
                # if not I assign a None value to beds_tmp
                # Assumption: if 'bed' is in the description of the item, it's a house, else it's a open plot
                if (elem.div.text.lower().find('bed')>=0):
                    beds_tmp = elem.div.text[0:elem.div.text.lower().find('bed')-1].strip()
                else:
                    beds_tmp = None

                # The below brings back a list of the nested div elements in elem
                # I then take the SECOND item in the list ('priceDescription'-element) via "[1]"
                # I leave the currency out via "[1:]"
                # I remove all the spaces, via .split() which returns a list of all remainig digits
                # I then join each list item to one another via .join (starting with a blank string)
                price_tmp = ''.join(elem.find_all('div')[1].text[1:].split())

                beds = int(beds_tmp) if ((beds_tmp is not None) and (beds_tmp.isnumeric())) else None
                price = int(price_tmp) if (price_tmp.isnumeric()) else None

                if ((title_tmp.find(area)>=0) and 
                    (counter <= 10) and 
                    (beds is not None) and 
                    (price is not None)):
                    init_list.append((area,price,beds))
                    counter += 1
                
    # The below is after all the areas data has been grabbed, it is put into a dataframe and summarised.
    area_df = pd.DataFrame(init_list,columns=['area','price','beds'])
    if (area_df.shape[0] > 0 ):
        a_gr_df = area_df.groupby('area').mean().reset_index()
        browser.close()
        return a_gr_df
    else:
        browser.close()
        return area_df

## Main code
# area_keys = list(region_dict.keys())
# region_dict_df = {}
# for area in area_keys:
#     print('Key Area: {}'.format(area))
#     area_df = propertyDataPopulator(region_dict[area][1:])
#     stg_dict = {area:area_df}
#     region_dict_df.update(stg_dict)
    
# region_dict_df[area_keys[0]].head()


##### Creating a checkpoint
In the interest of time I wrote both of the above outputs into JSON files. I had time constraints for two reasons:
1. Slow internet connection
2. Selenium was not setup headlessly for testing purposes

**Writing the files**

_(for data protection the writing files are named differently from the reading files; only if I am certain about the data to be written do I revert to the correct names below)_

In [ ]:
with open('/Users/Johan Strydom/AnacondaProjects/municipalReg.json','w') as file:
    json.dump(region_dict,file)
with open('/Users/Johan Strydom/AnacondaProjects/propertyDataF.json','w') as file2:
    file2.write(json.dumps({k:region_dict_df[k].to_json() for k in region_dict_df}))

**Reading the files**

Open the files and reuse the content through the following commands:

In [4]:
with open('/Users/Johan Strydom/AnacondaProjects/municipalRegions.json','r') as file:
    region_dict = json.load(file)
with open('/Users/Johan Strydom/AnacondaProjects/propertyDataFrames.json','r') as open_file:
    property_dict_df = json.load(open_file)
    
property_dfs = {k:pd.read_json(property_dict_df[k]).sort_index() for k in property_dict_df}
pd.options.display.float_format = '{:.2f}'.format  # set other global format

## Combining dataset - Clustering data prep
I first needed to create a single dataframe for each data set.

##### Control flow
The below creates the control flow dictionary to stitch both sets of data into a single dataframe at the appropriate level.

In [5]:
def controlDictColumnNames(cont_dict):
    new_dict = {}
    for k in cont_dict:
        if ('Main Place' in cont_dict[k]):
            new_col = 'Region'
            rename_col = 'Suburb'
        else:
            new_col = 'City'
            rename_col = 'Region'
        stg_dict = {k:(new_col,rename_col)}
        new_dict.update(stg_dict)
    return new_dict

cont_dict = {k:region_dict[k][0] for k in region_dict}
control_dict = controlDictColumnNames(cont_dict)

#### 1. Municipal region dataframe
I use the control_dict created above to inform the levels of each of the loose standing dictionaries. The below creates the summary dataframe for the municipal region data:

In [6]:
region_dfs = {}
for k in region_dict:
    new_col,rename_col = control_dict[k]
    stg_df = pd.DataFrame({k:region_dict[k][1:]})
    test_cols = list(stg_df.columns)[0]
    stg_df[new_col] = test_cols
    stg_df.rename(columns={test_cols:rename_col},inplace=True)
    new_cols = [stg_df.columns[1],stg_df.columns[0]]
    stg_df = stg_df[new_cols]
    stg_dict = {k:stg_df}
    region_dfs.update(stg_dict)

city_df = pd.DataFrame(columns=['City','Region'])
region_df = pd.DataFrame(columns=['Region','Suburb'])
for k in region_dfs:
    if ('City' in list(region_dfs[k].columns)):
        city_df = pd.concat([city_df,region_dfs[k]],ignore_index=True)
    else:
        region_df = region_df.append(region_dfs[k],ignore_index=True)

# city_df.reset_index(inplace=True)
city_shape = city_df.shape
region_shape = region_df.shape

municipal_df = city_df.merge(region_df,on='Region',how='left')
municipal_df.columns = list(map(lambda x: x.lower(),municipal_df.columns))
mun_shape = municipal_df.shape
print('City Shape: {}\nRegion Shape: {}\nMunicipal Shape: {}'.format(city_shape,region_shape,mun_shape))

City Shape: (117, 2)
Region Shape: (1374, 2)
Municipal Shape: (1442, 3)


##### Data dedup
The above shapes suggest that there is a duplication of rows on the join. Below is the investigation on the matter:

In [7]:
test_df = municipal_df.groupby(['region','city']).count().reset_index()
res_df = test_df.groupby(['region']).count().reset_index()
res_df.loc[res_df['suburb'] > 1][['region','city']]

,region,city
84,Rietfontein,2
86,Roodepoort,2


Rietfontein and Roodepoort are the two culprate regions.
1. **Roodepoort** -- This region is strictly a Johannesburg region, and technically is part of the Tshwane municipality as well, but is in a different smaller town. Therefore for the purpose of this investigation I have taken it out as a Tshwane city region.
2. **Rietfontein** -- This region is similar to Roodepoort, but flows the opposite logic, i.e. it is mainly referred to as a Tshwane suburb. The Rietfontein in Johannesburg, is in a different smaller town although it is technically part of the greater Johannesburg. I will remove it from the Johannesburg city.

In [8]:
munic_df = municipal_df.loc[~((municipal_df['region'] == 'Roodepoort') & (municipal_df['city'] == 'City of Tshwane')),:]
munic_df = munic_df.loc[~((munic_df['region'] == 'Rietfontein') & (municipal_df['city'] == 'City of Johannesburg')),:]
munic_df.shape

(1374, 3)

From the above shape the duplications have been removed.

#### 2. House sales dataframe
I can again use the control_dict to stitch the data on the appropriate level. I follow the same approach as for the above dataset with some tweaks on the process:

In [9]:
house_dfs = {}
for k in property_dfs:
    new_col,rename_col = control_dict[k]
    stg_df = property_dfs[k]
    init_stg_cols = list(stg_df.columns)
    stg_df[new_col] = k
    stg_df.rename(columns={'area':rename_col},inplace=True)
    new_cols = [new_col]
    new_cols[len(new_cols):] = list(stg_df.columns)[:-1]
    stg_df = stg_df[new_cols]
    lower_columns = list(stg_df.columns)
    stg_df.columns = list(map(lambda x: x.lower(),lower_columns))
    stg_dict = {k:stg_df}
    house_dfs.update(stg_dict)

city_df = pd.DataFrame(columns=['city','region','beds','price'])
region_df = pd.DataFrame(columns=['region','suburb','beds','price'])
for k in house_dfs:
    if ('city' in list(house_dfs[k].columns)):
        city_df = pd.concat([city_df,house_dfs[k]],ignore_index=True)
    else:
        region_df = region_df.append(house_dfs[k],ignore_index=True)

city_shape = city_df.shape
region_shape = region_df.shape

property_df = city_df.merge(region_df,on='region',how='left',suffixes=['_region','_suburb'])
prop_shape = property_df.shape
print('City Shape: {}\nRegion Shape: {}\nProperty Shape: {}'.format(city_shape,region_shape,prop_shape))

City Shape: (59, 4)
Region Shape: (517, 4)
Property Shape: (253, 7)


From the above it is clear that there is a massive loss of data when the city_df is joined to the region_df. This is because there is some regions or suburb names did not register on the sales site. From this point on, I will use the suburb data and only where it is missing, I will be using the regions data to fill in all the of the lower level suburbs.

### Combining Municipal and Sales data
I will be using the municipal data as my base and be joining the sales data onto it. As stated above, I will be using the individual city and region sales dataframes for the join in order to retain the most data.

In [10]:
## Creating the separate bases for the joining
reg_base_df = pd.DataFrame(columns=['city','region','suburb'])
reg_base_df = reg_base_df.append(munic_df)
city_base_df = pd.DataFrame(columns=['city','region','suburb'])
city_base_df = city_base_df.append(munic_df)

## Joining of the separate datasets with the two property_df's
reg_base_df = reg_base_df.merge(region_df,on=['region','suburb'],how='left')
city_base_df = city_base_df.merge(city_df,on=['city','region'],how='left')

## Filling the NaN values in ['beds','price'] with the appropriate values 
reg_base_df['beds'] = reg_base_df['beds'].fillna(city_base_df['beds'])
reg_base_df['price'] = reg_base_df['price'].fillna(city_base_df['price'])

## Checking the results
orig_shape = reg_base_df.shape
all_Nulls_shape = reg_base_df.loc[(reg_base_df['beds'].isnull()) & (reg_base_df['price'].isnull())].shape
null_price_shape = reg_base_df.loc[(reg_base_df['beds'].isnull()) & ~(reg_base_df['price'].isnull())].shape
null_beds_shape = reg_base_df.loc[~(reg_base_df['beds'].isnull()) & (reg_base_df['price'].isnull())].shape
no_nulls_shape = reg_base_df.loc[~(reg_base_df['beds'].isnull()) & ~(reg_base_df['price'].isnull())].shape
print('Original shape: {o}\nAll nulls: {a}\nPrice only nulls: {p}\nBeds only nulls: {b}\nNo nulls: {n}'.\
      format(o=orig_shape,a=all_Nulls_shape,p=null_price_shape,b=null_beds_shape,n=no_nulls_shape))

Original shape: (1374, 5)
All nulls: (357, 5)
Price only nulls: (0, 5)
Beds only nulls: (0, 5)
No nulls: (1017, 5)


From the above, the approach followed seemds to retain data well. There is however some regions where majority of its suburbs (+50%) are all populated with the regional average data. This posses a problem when using the data for clustering and will be addressed below.

In [11]:
comp_df = city_base_df.loc[~city_base_df['beds'].isnull()].groupby(['city','region']).mean()
comp_df.reset_index(inplace=True)
reg_base_df = reg_base_df.merge(comp_df,on=['city','region'],how='left',suffixes=['_act','_avg'])
reg_base_df['beds_act'] = reg_base_df['beds_act'].apply(lambda x: np.floor(x*100)/100)
reg_base_df['beds_avg'] = reg_base_df['beds_avg'].apply(lambda x: np.floor(x*100)/100)
reg_base_df['price_act'] = reg_base_df['price_act'].apply(np.floor)
reg_base_df['price_avg'] = reg_base_df['price_avg'].apply(np.floor)
reg_base_df['beds_comp'] = reg_base_df.apply(lambda row: 1 if (row['beds_act'] == row['beds_avg']) else 0 ,axis=1)
reg_base_df['price_comp'] = reg_base_df.apply(lambda row: \
                                              1 if (float(row['price_act']*100) == float(row['price_avg']*100)) \
                                              else 0 ,axis=1)
reg_base_df['overall_comp'] = reg_base_df.apply(lambda row: row['beds_comp']*row['price_comp'],axis=1)
reg_base_df.head(10)

,city,region,suburb,beds_act,price_act,beds_avg,price_avg,beds_comp,price_comp,overall_comp
0,City of Tshwane,Akasia,Amandasig,1.50,382500.00,3.00,1031727.00,0,0,0
1,City of Tshwane,Akasia,Chantelle,1.81,561600.00,3.00,1031727.00,0,0,0
2,City of Tshwane,Akasia,Clarina,1.00,332454.00,3.00,1031727.00,0,0,0
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000.00,3.00,1031727.00,0,0,0
4,City of Tshwane,Akasia,Heather View,3.00,1031727.00,3.00,1031727.00,1,1,1
5,City of Tshwane,Akasia,Hesteapark,3.00,1031727.00,3.00,1031727.00,1,1,1
6,City of Tshwane,Akasia,Karenpark,3.00,1031727.00,3.00,1031727.00,1,1,1
7,City of Tshwane,Akasia,Klerksoord,1.90,525500.00,3.00,1031727.00,0,0,0
8,City of Tshwane,Akasia,Ninapark,2.09,730963.00,3.00,1031727.00,0,0,0
9,City of Tshwane,Akasia,Onderstepoort Nature Reserve,3.00,1031727.00,3.00,1031727.00,1,1,1


In the above dataframe, it is clear to see which suburbs carry the average bed and price sales data for the region. What I have decided to do is to simply collapse all average carrying suburbs into a single entry, which has the region name as the suburb name.

I have also decided to drop all entries where no sales data is available.

In [12]:
reg_base_df.loc[reg_base_df['overall_comp'] == 1,['suburb']] = reg_base_df.loc[reg_base_df['overall_comp'] == 1]['region']
reg_base_df.drop_duplicates(inplace=True)
combined_df = reg_base_df.loc[~(reg_base_df['beds_act'].isnull()) & ~(reg_base_df['price_act'].isnull())].loc[:,'city':'price_act']
print(combined_df.shape)
combined_df.rename(columns={'beds_act':'beds','price_act':'price'},inplace=True)
combined_df.head()

(556, 5)


,city,region,suburb,beds,price
0,City of Tshwane,Akasia,Amandasig,1.50,382500.00
1,City of Tshwane,Akasia,Chantelle,1.81,561600.00
2,City of Tshwane,Akasia,Clarina,1.00,332454.00
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000.00
4,City of Tshwane,Akasia,Akasia,3.00,1031727.00


The final combined dataframe contains 556 entries of regions and suburbs with house sales data. This dataframe will now be used to get the latlongs for these suburbs. Thereafter the venue information for these locations will be retrieved.

## 3. LatLong data
The HERE geocoding API worked fantastically when populating the latlongs of the municipal regions in Johannesburg and Pretoria.

There is some good information that can be extracted from this API, but for this project I only used the latlong information.

The below is the function to extract the latlongs from HERE:

In [14]:
def getLatLongs(sch_txt):
    search_txt = sch_txt.replace(' ','%20')
    url = 'https://geocoder.api.here.com/6.2/geocode.json?app_id={h_id}&app_code={h_cde}&searchtext={stxt}'.\
        format(h_id=HERE_ID,h_cde=HERE_CODE,stxt=search_txt)
    response = requests.get(url).json()
    lat,lng = response['Response']['View'][0]['Result'][0]['Location']['DisplayPosition'].values()
    return (lat,lng)

Now that the function is written I need a search string to be created. This string combines suburb, region, city and finally the country (i.e. 'south africa').

In [15]:
def removeDuplicateWords(sentence):
    str_list = sentence.split()
    counts = dict(collections.Counter(x for x in str_list))
    remove = max(counts.items(), key=operator.itemgetter(1))
    if (remove[1]>1):
        new_sent = sentence.replace(remove[0],'',1).replace('  ',' ')
        final = new_sent
    else:
        final = sentence
    return final.lower()

combined_df['search_string'] = combined_df.apply(lambda r: \
                                                 r.suburb+' '+\
                                                 r.region+' '+\
                                                 ('Pretoria' if ('Tshwane' in r.city) else 'Johannesburg')+\
                                                 ' south africa'\
                                                 ,axis=1)
combined_df['search_string'] = combined_df.apply(lambda r:\
                                                 removeDuplicateWords(r.search_string)\
                                                 ,axis=1)
combined_df.loc[(combined_df['city'] == 'City of Johannesburg') & (combined_df['region'] == 'Alexandra')].head()
# combined_df.head(10)

,city,region,suburb,beds,price,search_string
578,City of Johannesburg,Alexandra,Alexandra,3.00,850000.00,alexandra johannesburg south africa
645,City of Johannesburg,Alexandra,East Bank,3.25,808750.00,east bank alexandra johannesburg south africa


Now to combine the both of the inputs into the single dataframe:

In [16]:
# combined_df['latlong'] = combined_df.apply(lambda r: getLatLongs(r.search_string),axis=1)
combined_df.head()

,city,region,suburb,beds,price,search_string
0,City of Tshwane,Akasia,Amandasig,1.50,382500.00,amandasig akasia pretoria south africa
1,City of Tshwane,Akasia,Chantelle,1.81,561600.00,chantelle akasia pretoria south africa
2,City of Tshwane,Akasia,Clarina,1.00,332454.00,clarina akasia pretoria south africa
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000.00,heatherdale ah akasia pretoria south africa
4,City of Tshwane,Akasia,Akasia,3.00,1031727.00,akasia pretoria south africa


##### Creating check point

**_Writing dataframe to file_**

_(for data protection reasons, the below name is changed)_

In [ ]:
with open('/Users/Johan Strydom/AnacondaProjects/combinedRegionsSalesL.json','w') as check3:
    check3.write(json.dumps(combined_df.to_json()))

**_Reading from file_**

In [17]:
with open('/Users/Johan Strydom/AnacondaProjects/combinedRegionsSalesLatlongs.json') as check3:
    combined_df = json.load(check3)

combined_df = pd.read_json(combined_df).sort_index()
col_in_order = ['city','region','suburb','beds','price','search_string','latlong']
combined_df = combined_df[col_in_order]
combined_df['latlong'] = combined_df.apply(lambda r: tuple(r.latlong),axis=1)
combined_df.head()

,city,region,suburb,beds,price,search_string,latlong
0,City of Tshwane,Akasia,Amandasig,1.50,382500,amandasig akasia pretoria south africa,"(-25.67387, 28.10067)"
1,City of Tshwane,Akasia,Chantelle,1.81,561600,chantelle akasia pretoria south africa,"(-25.66581, 28.0923)"
2,City of Tshwane,Akasia,Clarina,1.00,332454,clarina akasia pretoria south africa,"(-25.64972, 28.11789)"
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,heatherdale ah akasia pretoria south africa,"(-25.66769, 28.1204)"
4,City of Tshwane,Akasia,Akasia,3.00,1031727,akasia pretoria south africa,"(-25.65923, 28.10318)"


In [18]:
over_shape = combined_df.shape
group_shape = combined_df.groupby(['latlong']).count().shape
print('Overall shape: {}\nGrouped shape: {}'.format(over_shape,group_shape))

Overall shape: (556, 7)
Grouped shape: (547, 6)


In [19]:
test_df = combined_df.groupby(['latlong']).count().reset_index()
ll_lst = list(test_df.loc[test_df['city'] > 1]['latlong'])
combined_df.loc[combined_df['latlong'].isin(ll_lst)].sort_values(['latlong'],axis=0)

,city,region,suburb,beds,price,search_string,latlong
955,City of Johannesburg,Lakeside,Lakeside,2.66,1659444,lakeside johannesburg south africa,"(-26.10003, 28.14939)"
821,City of Johannesburg,Johannesburg,Lakeside,2.66,1659444,lakeside johannesburg south africa,"(-26.10003, 28.14939)"
1227,City of Johannesburg,Sandton,Hurlingham Gardens,3.50,5425000,hurlingham gardens sandton johannesburg south ...,"(-26.09095, 28.02361)"
1226,City of Johannesburg,Sandton,Hurlingham,2.36,2341726,hurlingham sandton johannesburg south africa,"(-26.09095, 28.02361)"
1079,City of Johannesburg,Randburg,Johannesburg North,2.18,1190636,north randburg johannesburg south africa,"(-26.0285, 27.97593)"
1052,City of Johannesburg,Randburg,Bosmont,3.36,932727,bosmont randburg johannesburg south africa,"(-26.0285, 27.97593)"
1043,City of Johannesburg,Randburg,Randburg,1.00,320000,randburg johannesburg south africa,"(-26.0285, 27.97593)"
657,City of Johannesburg,Chartwell,Chartwell,4.18,3418181,chartwell johannesburg south africa,"(-25.99462, 27.98145)"
656,City of Johannesburg,Chartwell,Chartwell AH,6.00,3750000,ah chartwell johannesburg south africa,"(-25.99462, 27.98145)"
361,City of Tshwane,Pretoria,Lynnwood Manor,2.54,1298999,lynnwood manor pretoria south africa,"(-25.76484, 28.26792)"


From the above it is clear that there are some duplicate latlongs. However these places are geographically very close to each other, and therefore similar latlongs does not harm the outcome of the project. There are two considerations for this:
1. The latlongs will be used for the **FourSquare** API, where the surrounding venues are extracted, these places would have had very similar venue
2. There are very few duplications (only 9 combinations, influencing 17 places)

Before we progress, I need to separate the latlong tuple into two separate columns: 'lat' and 'lng'

In [20]:
combined_df['lat'] = [x[0] for x in combined_df['latlong']]
combined_df['lng'] = [x[1] for x in combined_df['latlong']]
combined_df.head()

,city,region,suburb,beds,price,search_string,latlong,lat,lng
0,City of Tshwane,Akasia,Amandasig,1.50,382500,amandasig akasia pretoria south africa,"(-25.67387, 28.10067)",-25.67,28.10
1,City of Tshwane,Akasia,Chantelle,1.81,561600,chantelle akasia pretoria south africa,"(-25.66581, 28.0923)",-25.67,28.09
2,City of Tshwane,Akasia,Clarina,1.00,332454,clarina akasia pretoria south africa,"(-25.64972, 28.11789)",-25.65,28.12
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,heatherdale ah akasia pretoria south africa,"(-25.66769, 28.1204)",-25.67,28.12
4,City of Tshwane,Akasia,Akasia,3.00,1031727,akasia pretoria south africa,"(-25.65923, 28.10318)",-25.66,28.10


_\*Note: dataframes' display options have been changed to only show upto two decimals, but the data is still in tact_

We are now ready to proceed to the final stage of data aqcuistion.

## 4. Surrounding venues (FourSquare)
For South Africa the FourSquare data is a bit more sparse, but not non-existent. I have decided to still keep the suburbs where there is no FourSquare data available, as this is also informative about the specific suburb.

When populating the FourSquare data, I wrote a little progress bar to give feedback on the progress.

In [21]:
class ProgressBar:
    def __init__(self):
        self.progress = 0
        
    def updateProgress(self,curr,total):
        perc = curr / total
        prog = math.floor(perc * 20)
        if (self.progress < prog):
            self.progress = prog
            clear_output()
            print('ProgressBar: <{:<20}> [{:4.0f}%]'.format('='*self.progress,math.floor(perc*100)))
        else:
            pass

In [22]:
def getNearbyVenues(citys, regions, suburbs, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    total = suburbs.shape[0]
    counter = 0
    progBar = ProgressBar()
    progBar.updateProgress(counter,total)
    for city,region,suburb,lat,lng in zip(citys, regions, suburbs, latitudes, longitudes):
        counter += 1
        progBar.updateProgress(counter,total)
#         print(str(counter) + ": " + nhood)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                city,
                region,
                suburb,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
            progBar.updateProgress(counter,total)
    #         progressBar(counter,total)
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city',
                             'region',
                              'suburb',
                              'suburb_lat', 
                              'suburb_lng', 
                              'venue', 
                              'venue_lat', 
                              'venue_lng', 
                              'venue_category']
    
    print(('Shape of the returned dataframe is: {}').format(nearby_venues.shape))
    
    return(nearby_venues)

In [ ]:
# venue_df = getNearbyVenues(combined_df['city'],\
#                           combined_df['region'],\
#                           combined_df['suburb'],\
#                           combined_df['lat'],\
#                           combined_df['lng'])

##### Check point 4
**_Writing the Venue dataframe_**

In [23]:
with open('/Users/Johan Strydom/AnacondaProjects/fourSquareV.json','w') as check4:
    check4.write(json.dumps(venue_df.to_json()))

NameError: name 'venue_df' is not defined

**_Reading the Venue dataframe_**

In [24]:
with open('/Users/Johan Strydom/AnacondaProjects/fourSquareVenueData.json','r') as check4:
    venue_df = json.load(check4)

venue_df = pd.read_json(venue_df).sort_index()
col_in_order = ['city','region','suburb','suburb_lat','suburb_lng',\
                'venue','venue_lat','venue_lng','venue_category']
venue_df = venue_df[col_in_order]
venue_df.head()

,city,region,suburb,suburb_lat,suburb_lng,venue,venue_lat,venue_lng,venue_category
0,City of Tshwane,Akasia,Chantelle,-25.67,28.09,Wolmer,-25.67,28.09,Neighborhood
1,City of Tshwane,Akasia,Heatherdale AH,-25.67,28.12,Debonairs Pizza,-25.67,28.12,Pizza Place
2,City of Tshwane,Akasia,Akasia,-25.66,28.10,Nasi goreng jakarta,-25.66,28.10,Asian Restaurant
3,City of Tshwane,Akasia,Ninapark,-25.68,28.14,Ninapark Shopping Centre,-25.68,28.14,Shopping Mall
4,City of Tshwane,Akasia,Ninapark,-25.68,28.14,Cash Converters Nina Park,-25.68,28.15,Pawn Shop


## Final Dataframe for Clustering
Below all the data collected is combined into a single dataframe

In [25]:
cluster_df = combined_df.merge(venue_df,on=['city','region','suburb'],how='left')
print(cluster_df.shape)
cluster_df.head()

(2430, 15)


,city,region,suburb,beds,price,search_string,latlong,lat,lng,suburb_lat,suburb_lng,venue,venue_lat,venue_lng,venue_category
0,City of Tshwane,Akasia,Amandasig,1.50,382500,amandasig akasia pretoria south africa,"(-25.67387, 28.10067)",-25.67,28.10,nan,nan,NaN,nan,nan,NaN
1,City of Tshwane,Akasia,Chantelle,1.81,561600,chantelle akasia pretoria south africa,"(-25.66581, 28.0923)",-25.67,28.09,-25.67,28.09,Wolmer,-25.67,28.09,Neighborhood
2,City of Tshwane,Akasia,Clarina,1.00,332454,clarina akasia pretoria south africa,"(-25.64972, 28.11789)",-25.65,28.12,nan,nan,NaN,nan,nan,NaN
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,heatherdale ah akasia pretoria south africa,"(-25.66769, 28.1204)",-25.67,28.12,-25.67,28.12,Debonairs Pizza,-25.67,28.12,Pizza Place
4,City of Tshwane,Akasia,Akasia,3.00,1031727,akasia pretoria south africa,"(-25.65923, 28.10318)",-25.66,28.10,-25.66,28.10,Nasi goreng jakarta,-25.66,28.10,Asian Restaurant


This dataframe's columns are cleaned up, which will then be used in cluster pre-processing:

In [26]:
final_cols = ['city','region','suburb','beds','price','lat','lng','venue','venue_lat','venue_lng','venue_category']
final_df = cluster_df[final_cols]
final_df.head()

,city,region,suburb,beds,price,lat,lng,venue,venue_lat,venue_lng,venue_category
0,City of Tshwane,Akasia,Amandasig,1.50,382500,-25.67,28.10,NaN,nan,nan,NaN
1,City of Tshwane,Akasia,Chantelle,1.81,561600,-25.67,28.09,Wolmer,-25.67,28.09,Neighborhood
2,City of Tshwane,Akasia,Clarina,1.00,332454,-25.65,28.12,NaN,nan,nan,NaN
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,-25.67,28.12,Debonairs Pizza,-25.67,28.12,Pizza Place
4,City of Tshwane,Akasia,Akasia,3.00,1031727,-25.66,28.10,Nasi goreng jakarta,-25.66,28.10,Asian Restaurant


##### Check point 5

**_Writing final dataframe_**

In [ ]:
with open('/Users/Johan Strydom/AnacondaProjects/finalCombinedD.json','w') as check5:
    check5.write(json.dumps(final_df.to_json()))

**_Reading dataframe_**

In [27]:
with open('/Users/Johan Strydom/AnacondaProjects/finalCombinedDataFrame.json','r') as check4:
    final_df = json.load(check4)

final_df = pd.read_json(final_df).sort_index()
col_in_order = ['city','region','suburb','beds','price','lat','lng',\
                'venue','venue_lat','venue_lng','venue_category']
final_df = final_df[col_in_order]

In [28]:
final_df.head()

,city,region,suburb,beds,price,lat,lng,venue,venue_lat,venue_lng,venue_category
0,City of Tshwane,Akasia,Amandasig,1.50,382500,-25.67,28.10,None,nan,nan,None
1,City of Tshwane,Akasia,Chantelle,1.81,561600,-25.67,28.09,Wolmer,-25.67,28.09,Neighborhood
2,City of Tshwane,Akasia,Clarina,1.00,332454,-25.65,28.12,None,nan,nan,None
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,-25.67,28.12,Debonairs Pizza,-25.67,28.12,Pizza Place
4,City of Tshwane,Akasia,Akasia,3.00,1031727,-25.66,28.10,Nasi goreng jakarta,-25.66,28.10,Asian Restaurant


## Clustering pre-processing
The data is now to be prepocessed in the following manner:
1. **Normalising** - the columns 'beds' and 'price' need to be normalised
2. **Dummy fields** - the venue_category column needs to be split into multiple dummy fields

Once the above two steps are completed, the cluster model can be run.
### 1. Normalizing
In order to not create inherrent biases on the data, fields that have different orders of magnetude are reduced to a number between 0 and 1.

The normalization of beds and price follows:

In [29]:
## Setting up the environment
from sklearn import preprocessing

## 1. Normalizing 'beds'
x = final_df[['beds']].values #returns a numpy array
beds_scaler = preprocessing.MinMaxScaler()
x_scaled = beds_scaler.fit_transform(x)
final_df['beds_norm'] = pd.DataFrame(x_scaled)

## 2. Normalizing 'price'
y = final_df[['price']].values
price_scaler = preprocessing.MinMaxScaler()
y_scaled = price_scaler.fit_transform(y)
final_df['price_norm'] = pd.DataFrame(y_scaled)

print(final_df.shape)
final_df.head()[['city','region','suburb','beds','price','beds_norm','price_norm']]

(2430, 13)


C:\Users\Johan Strydom\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,city,region,suburb,beds,price,beds_norm,price_norm
0,City of Tshwane,Akasia,Amandasig,1.50,382500,0.03,0.01
1,City of Tshwane,Akasia,Chantelle,1.81,561600,0.04,0.01
2,City of Tshwane,Akasia,Clarina,1.00,332454,0.00,0.01
3,City of Tshwane,Akasia,Heatherdale AH,5.66,4309000,0.25,0.15
4,City of Tshwane,Akasia,Akasia,3.00,1031727,0.11,0.03


### 2. Dummy fields
Categorical variables can be included in clustering models, but they need to be encoded into dummy variables in order to be number crunched. This functionality is built into the pandas library

In [30]:
venues_encoded = pd.get_dummies(final_df[['venue_category']], prefix="", prefix_sep="")
venues_encoded[['city','region','suburb']] = final_df[['city','region','suburb']] 
ordered_cols = ['city','region','suburb'] + list(venues_encoded.columns[:243])
venues_encoded = venues_encoded[ordered_cols]
venues_encoded.head()

,city,region,suburb,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,City of Tshwane,Akasia,Amandasig,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,City of Tshwane,Akasia,Chantelle,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,City of Tshwane,Akasia,Clarina,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,City of Tshwane,Akasia,Heatherdale AH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,City of Tshwane,Akasia,Akasia,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In order to simplify any following calculations I am creating a single 'key' row, which will be used for furhter joining. This needs to happen on both applicable dataframes (i.e. **_"venues encoded"_** and **_"final df"_**).

In [31]:
venues_encoded['key'] = venues_encoded.apply(lambda r:\
                                             r.city+'-'+\
                                             r.region+'-'+\
                                             r.suburb\
                                            ,axis=1)
final_df['key'] = final_df.apply(lambda r:\
                                 r.city+'-'+\
                                 r.region+'-'+\
                                 r.suburb\
                                ,axis=1)
ven_cols = [venues_encoded.columns[-1]] + list(venues_encoded.columns[:-1])
fin_cols = [final_df.columns[-1]] + list(final_df.columns[:-1])
venues_encoded = venues_encoded[ven_cols]
final_df = final_df[fin_cols]

The final step is to create a dataframe that only has the key groupings:

In [32]:
ven_sel = list(venues_encoded.columns)[4:]
fin_sel = ['key','beds_norm','price_norm']
comb_sel = fin_sel + ven_sel
mean_df = final_df.merge(venues_encoded,on='key',how='inner')[comb_sel]
cluster_df = mean_df.groupby(['key']).mean().reset_index()
print(cluster_df.shape)
cluster_df.head()

(556, 246)


,key,beds_norm,price_norm,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,City of Johannesburg-Alexandra-Alexandra,0.11,0.02,0.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,City of Johannesburg-Alexandra-East Bank,0.12,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,City of Johannesburg-Chartwell-Chartwell,0.17,0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,City of Johannesburg-Chartwell-Chartwell AH,0.27,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,City of Johannesburg-Dainfern-Dainfern,0.03,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


From the shape of cluster_df it is clear to see that there was no data loss during processing up to this stag. It is evident from the shape of the combined_df (listing all final suburbs) and the final cluster_df (listing all final grouped venues) have the same number of rows.

**We are therefore now ready proceed to the clustering!**

\* *One __last__ thing* on the venues_encoded dataframe. I will be creating a separate dataframe which has the sole purpose of showing which are the top ten venues (w.r.t. number) in the surrounding area.

In [33]:
ven_sel = ['key'] + list(venues_encoded.columns)[4:]
top_raw = cluster_df[ven_sel]

def most_common_venues(row):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:10]

col_numbering = ['st', 'nd', 'rd']

columns = ['key']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, col_numbering[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

venues_orderd = pd.DataFrame(columns=columns)
venues_orderd['key'] = top_raw['key']

for ind in np.arange(cluster_df.shape[0]):
    venues_orderd.iloc[ind, 1:] = most_common_venues(top_raw.iloc[ind, :])

venues_orderd.head()

final_group_df = final_df[['key','city','region','suburb','beds','price','lat','lng']].groupby(['key','city','region','suburb']).mean().reset_index()

display_col_order = ['key','city','region','suburb','beds','price','lat','lng',\
                     '1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue',\
                     '5th Most Common Venue','6th Most Common Venue','7th Most Common Venue','8th Most Common Venue',\
                     '9th Most Common Venue','10th Most Common Venue']
display_df = venues_orderd.merge(final_group_df,on='key',how='inner')
display_df = display_df[display_col_order]
print(display_df.shape)
display_df.head(3)

(556, 18)


,key,city,region,suburb,beds,price,lat,lng,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of Johannesburg-Alexandra-Alexandra,City of Johannesburg,Alexandra,Alexandra,3.00,850000,-26.11,28.10,Afghan Restaurant,Zoo,Fast Food Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,City of Johannesburg-Alexandra-East Bank,City of Johannesburg,Alexandra,East Bank,3.25,808750,-26.10,28.11,Soccer Field,Zoo,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
2,City of Johannesburg-Chartwell-Chartwell,City of Johannesburg,Chartwell,Chartwell,4.18,3418181,-25.99,27.98,Garden Center,Deli / Bodega,Coffee Shop,Farm,Café,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


# Clustering
I will be running the KMeans clustering method. A difficult decision to make is how many clusters will be appropriate. It is also a highly subjective matter.

Given the wide geographical area and the very diverse populations being brought into play, I do believe there needs to be a larger than normal number of clusters. In order to make the result simple to display on a map and clear out confusion, it cannot be too many either.

I have decided on 10.

_(**Note**: I did test on 20 and 15, however 10 seemed to produce the most clarifying map for visualization)_

In [34]:
# set number of clusters
kclusters = 10
cluster_arr = cluster_df.drop('key', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_arr)

## Adding the cluster number to the beginning of display_df
if 'cluster_num' in display_df.columns:
    display_df['cluster_num'] = kmeans.labels_
else:
    display_df.insert(0,'cluster_num',kmeans.labels_)
display_df.head()

,cluster_num,key,city,region,suburb,beds,price,lat,lng,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,City of Johannesburg-Alexandra-Alexandra,City of Johannesburg,Alexandra,Alexandra,3.00,850000,-26.11,28.10,Afghan Restaurant,Zoo,Fast Food Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,6,City of Johannesburg-Alexandra-East Bank,City of Johannesburg,Alexandra,East Bank,3.25,808750,-26.10,28.11,Soccer Field,Zoo,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
2,6,City of Johannesburg-Chartwell-Chartwell,City of Johannesburg,Chartwell,Chartwell,4.18,3418181,-25.99,27.98,Garden Center,Deli / Bodega,Coffee Shop,Farm,Café,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
3,6,City of Johannesburg-Chartwell-Chartwell AH,City of Johannesburg,Chartwell,Chartwell AH,6.00,3750000,-25.99,27.98,Garden Center,Deli / Bodega,Coffee Shop,Farm,Café,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
4,6,City of Johannesburg-Dainfern-Dainfern,City of Johannesburg,Dainfern,Dainfern,1.63,588617,-25.98,28.00,Soccer Field,Animal Shelter,Golf Course,Zoo,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


In [35]:
display_df.groupby('cluster_num').count()[['suburb']].sort_values(['suburb'],ascending=False)

,suburb
cluster_num,
6,363
0,57
2,49
5,43
4,13
1,8
8,8
9,6
7,5



# Mapping the clusters
Now to display the result of the work. This first map separates the points into their overall regions. The second map shows the resulting clusters.

#### Map1: Regional mapping

In [36]:
## Gauteng center
latitude, longitude = (-25.941705, 28.136774)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

colors_array = cm.rainbow(np.linspace(0, 1, display_df['region'].nunique()))
rainbow = [colors.rgb2hex(i) for i in colors_array]
region_list = list(display_df['region'].unique())

# add markers to the map
markers_colors = []
for lat, lon, key, cluster, reg in zip(display_df['lat'], display_df['lng'],\
                                  display_df['key'], display_df['cluster_num'],\
                                  display_df['region']):
    label = folium.Popup(str(key) + ' [' + str(cluster) +']', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[region_list.index(reg)-1],
        fill=True,
        fill_color=rainbow[region_list.index(reg)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Map2: Cluster mapping

In [37]:
## Gauteng center
latitude, longitude = (-25.941705, 28.136774)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, key, cluster in zip(display_df['lat'], display_df['lng'], display_df['key'], display_df['cluster_num']):
    label = folium.Popup(str(key) + ' [' + str(cluster) + ']', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Conclusion
### Findings
Finally mapping the dataset does do the effort justice. A quick few comments on the end result:
1. Overall there is a strong similarity for the entire geographical area. There is an evident majority cluster (cluster = 6), which does make sense as both of these cities are considered to be the heart of Gauteng.
2. There does seem to be a modest difference in the homogeny between Pretoria and Johannesburg, when focusing in closer. Johannesburg does seem to be a bit more metropolitan in the sense that it carries more diverse clusters over a smaller geographical area.
3. The space between the two cities (called Midrand), also has a strong heterogenic appearance. This could be because some of the more progressive communities have already moved to the centre, which is closer either way.

### Audience feedback
From the analysis performed it does seem that more care needs to be taken to move from Pretoria to Johannesburg, if your objective is to relocate to a similar neighbourhood. The differences between the separate suburbs will be more pronounced and happen at a much less gradual scale.

However if the goal is to feel part of a greater society, where you feel that you are part of a melting pot of cultures, then it may be precisely what you are looking for.

Ultimately there are more similar areas than not. It comes down to the choice of lifestyle that you are wanting to follow.

Safe stay, or happy hopping.

**_PS._** for your convenience a list of all the areas are presented below. If you would like to see to which cluster your neighbourhood / suburb prescribes to please read below.

##### Search text

In [ ]:
search_suburb = 'Florida'

def similarSuburbs(clust_num):
    sim_list = list(display_df.loc[display_df['cluster_num'] == clust_num][0:10]['suburb'])
    return sim_list

def checkList(s_txt):
    sub_list = list(display_df['suburb'].unique())
    if s_txt in sub_list:
        c_num = display_df.loc[display_df['suburb'] == s_txt]['cluster_num'].values[0]
        others = [s_txt] + similarSuburbs(c_num)
        print(display_df.loc[display_df['suburb'].isin(others)][['cluster_num','suburb','beds','price']])
    else:
        print('The suburb requested was not found.\nIt may be that the spelling does not correspond to our list.')

checkList(search_suburb)

##### List

In [ ]:
list(display_df['suburb'].groupby(display_df['cluster_num']))